In [8]:
"""
Discriminator and Generator implementation from DCGAN paper,
with removed Sigmoid() as output from Discriminator (and therefor
it should be called critic)

Programmed by Aladdin Persson <aladdin.persson at hotmail dot com>
* 2020-11-01: Initial coding
* 2022-12-20: Small revision of code, checked that it works with latest PyTorch version
"""

import torch
import torch.nn as nn


class Discriminator(nn.Module):
    def __init__(self, channels_img, features_d):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            # input: N x channels_img x 64 x 64
            nn.Conv2d(
                channels_img, features_d, kernel_size=4, stride=2, padding=1
            ),
            nn.LeakyReLU(0.2),
            # _block(in_channels, out_channels, kernel_size, stride, padding)
            self._block(features_d, features_d * 2, 4, 2, 1),
            self._block(features_d * 2, features_d * 4, 4, 2, 1),
            self._block(features_d * 4, features_d * 8, 4, 2, 1),
            # After all _block img output is 4x4 (Conv2d below makes into 1x1)
            nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0),
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,
            ),
            nn.InstanceNorm2d(out_channels, affine=True),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.disc(x)


class Generator(nn.Module):
    def __init__(self, channels_noise, channels_img, features_g):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            # Input: N x channels_noise x 1 x 1
            self._block(channels_noise, features_g * 16, 4, 1, 0),  # img: 4x4
            self._block(features_g * 16, features_g * 8, 4, 2, 1),  # img: 8x8
            self._block(features_g * 8, features_g * 4, 4, 2, 1),  # img: 16x16
            self._block(features_g * 4, features_g * 2, 4, 2, 1),  # img: 32x32
            nn.ConvTranspose2d(
                features_g * 2, channels_img, kernel_size=4, stride=2, padding=1
            ),
            # Output: N x channels_img x 64 x 64
            nn.Tanh(),
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.net(x)


def initialize_weights(model):
    # Initializes weights according to the DCGAN paper
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)


def test():
    N, in_channels, H, W = 8, 3, 64, 64
    noise_dim = 100
    x = torch.randn((N, in_channels, H, W))
    disc = Discriminator(in_channels, 8)
    assert disc(x).shape == (N, 1, 1, 1), "Discriminator test failed"
    gen = Generator(noise_dim, in_channels, 8)
    z = torch.randn((N, noise_dim, 1, 1))
    assert gen(z).shape == (N, in_channels, H, W), "Generator test failed"
    print("Success, tests passed!")

if __name__ == "__main__":
    test()

Success, tests passed!


In [9]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn import metrics
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim as optim
import math
import random
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import StratifiedKFold

In [10]:
import glob

path = r"/home/jupyter/src/Thesis_Project/Data_padded/100"
all_files = glob.glob(path + "/*")
all_files.sort()

In [11]:
total = 40000

def extract_samples(target):
    stop = 0
    positive_data = None
    negative_data = None

    positive_init = False
    negative_init = False

    positive_indices = None
    negative_indices = None

    print('**********************', target)
    for i, filename in enumerate(all_files):
        if i == target:
            positive_dataset = np.load(filename)
            positive_indices = list(range(len(positive_dataset)))
            np.random.shuffle(positive_indices)
            positive_init = True
            positive_data = positive_dataset[positive_indices]
            # print("Current positive keystroke images Data shape is",positive_data.shape)

    positive_length = len(positive_data)
    negative_length = total - positive_length

    for i, filename in enumerate(all_files):
        
        if i != target and stop < 30:
            negative_dataset = np.load(filename)
            nega_len = len(negative_dataset)
            smaple_len = 1
            if random.randint(0, 1) == 0:
                smaple_len = math.floor(nega_len/negative_length * positive_length)
            else:
                smaple_len = math.ceil(nega_len/negative_length * positive_length)
            negative_indices = list(range(nega_len))

            if not negative_init:
                negative_data = negative_dataset[negative_indices[:smaple_len]]
                negative_init = True
            else:
                extend_sameple = negative_dataset[negative_indices[:smaple_len]]
                negative_data = np.concatenate((negative_data, extend_sameple), axis=0)
                stop += 1

            # print("Current negative keystroke images Data shape is", negative_data.shape)
            
      
    
        

    print("Finaly positive keystroke images Data shape is",positive_data.shape)
    print("Finaly negative keystroke images Data shape is", negative_data.shape)

    return positive_data, negative_data

In [12]:
class KeystrokesDataset(Dataset):
    def __init__(self, samples, labels, transform):
        self.samples = samples
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        x = self.samples[idx]
        x = x.transpose((1, 2 ,0))
        x = self.transform(image=x)['image']
        label = torch.from_numpy(self.labels[idx]).float()
        return x, label


In [13]:
def encapsulate(positive_data, negative_data):

    train_dataloaders = []
    test_dataloaders = []
  
    # try and use Cross Fold Val
    skf = StratifiedKFold(n_splits=5, random_state=None, shuffle=True)

    total_dataset = np.concatenate((positive_data, negative_data), axis = 0)
    total_labels = np.concatenate((
      np.ones((len(positive_data), 1)), 
      np.zeros((len(negative_data), 1))
    ), axis=0)

    for train_index, test_index in skf.split(total_dataset, total_labels):
        train_dataset_numpy, test_dataset_numpy = total_dataset[train_index], total_dataset[test_index]
        train_labels_numpy, test_labels_numpy = total_labels[train_index], total_labels[test_index]

        mean = 0.
        std = 0.

        batch_samples = len(train_dataset_numpy) # batch size (the last batch can have smaller size!)
        images = train_dataset_numpy.reshape(batch_samples, 5, -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)

        mean /= len(train_dataset_numpy)
        std /= len(train_dataset_numpy)

    # transform = transforms.Compose([
    #   transforms.Normalize(mean=mean,std=std)
    # ])

    train_transform = A.Compose([
        A.Normalize(mean=mean.tolist(), std=std.tolist(), max_pixel_value=1.0, p=1.0),
        A.CoarseDropout(p=0.5, max_height=3, max_width=3, fill_value=0),
        # A.Cutout(p=0.5, max_h_size=3, max_w_size=3, fill_value=0),
        ToTensorV2(p=1.0),
    ], p=1.)

    test_transform = A.Compose([
        A.Normalize(mean=mean.tolist(), std=std.tolist(), max_pixel_value=1.0, p=1.0),
        ToTensorV2(p=1.0),
    ], p=1.)


    train_dataset = KeystrokesDataset(train_dataset_numpy, train_labels_numpy, train_transform)
    test_dataset = KeystrokesDataset(test_dataset_numpy, test_labels_numpy, test_transform)

    train_dataloader = DataLoader(train_dataset, batch_size=20, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=20, shuffle=True)

    train_dataloaders.append(train_dataloader)
    test_dataloaders.append(test_dataloader)

    return train_dataloaders, test_dataloaders

In [14]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
TF_CPP_MIN_LOG_LEVEL="2"

device = torch.device("cuda"if torch.cuda.is_available() else "cpu")

LEARNING_RATE = 5e-5
BATCH_SIZE = 64
IMAGE_SIZE = 64
CHANNELS_IMG = 5
Z_DIM = 128
NUM_EPOCHS = 50
FEATURES_CRITIC = 64
FEATURES_GEN = 64
CRITIC_ITERATIONS = 10
WEIGHT_CLIP = 0.01


transforms = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(
            [.5 for _ in range(CHANNELS_IMG)], [.5 for _ in range(CHANNELS_IMG)]),
    ]
)


def train_GAN(dataloader,MODEL_NUM):

    # initialize gen and disc/critic
    gen = Generator(Z_DIM, CHANNELS_IMG, FEATURES_GEN).to(device)
    critic = Discriminator(CHANNELS_IMG, FEATURES_CRITIC).to(device)
    initialize_weights(gen)
    initialize_weights(critic)

    # initializate optimizer
    opt_gen = optim.RMSprop(gen.parameters(), lr=LEARNING_RATE)
    opt_critic = optim.RMSprop(critic.parameters(), lr=LEARNING_RATE)

    # for tensorboard plotting
    fixed_noise = torch.randn(32, Z_DIM, 1, 1).to(device)
    writer_real = SummaryWriter(f"logs/real")
    writer_fake = SummaryWriter(f"logs/fake")
    step = 0
    stop = False

    gen.train()
    critic.train()

    for epoch in range(NUM_EPOCHS):
        # Target labels not needed! <3 unsupervised
        for batch_idx, (data, _) in enumerate(tqdm(dataloader[0])):
            data = data.to(device)
            cur_batch_size = data.shape[0]

            # Train Critic: max E[critic(real)] - E[critic(fake)]
            for _ in range(CRITIC_ITERATIONS):
                noise = torch.randn(BATCH_SIZE, Z_DIM, 1, 1).to(device)
                fake = gen(noise)
                critic_real = critic(data).reshape(-1)
                critic_fake = critic(fake).reshape(-1)
                loss_critic = -(torch.mean(critic_real) - torch.mean(critic_fake))
                critic.zero_grad()
                loss_critic.backward(retain_graph=True)
                opt_critic.step()

                # clip critic weights between -0.01, 0.01
                for p in critic.parameters():
                    p.data.clamp_(-WEIGHT_CLIP, WEIGHT_CLIP)

            # Train Generator: max E[critic(gen_fake)] <-> min -E[critic(gen_fake)]
            gen_fake = critic(fake).reshape(-1)
            loss_gen = -torch.mean(gen_fake)
            gen.zero_grad()
            loss_gen.backward()
            opt_gen.step()

            # Print losses occasionally and print to tensorboard
            if batch_idx % 100 == 0 and batch_idx > 0:
                gen.eval()
                critic.eval()
                # print(
                #     f"Epoch [{epoch}/{NUM_EPOCHS}] Batch {batch_idx}/{len(dataloader)} \
                #       Loss D: {loss_critic:.4f}, loss G: {loss_gen:.4f}"
                # )
                step += 1
                gen.train()
                critic.train()
            
            if epoch + 1 == NUM_EPOCHS and not stop:
                generated_samples = gen(noise).cpu().detach().numpy()
                samples = list()
                c = list()
                for channel in generated_samples:
                    for rows in range(len(channel)):
                        r = list()
                        for row in range(len(channel[rows])):
                            r.append(channel[rows][row][11:53])
                        r = r[11:53]
                        c.append(r)
                    samples.append(c)
                    c = list()
                samples = np.array(samples)
                np.save(r"/home/jupyter/src/Thesis_Project/WGAN_data/100/gen_data_"+MODEL_NUM+".npy",samples)
                print(samples.shape,"HERE")
                stop = True
        
for i in range(23,75):
    data = extract_samples(i)
    dataloader,dataloader2 = encapsulate(data[0],data[1])
    train_GAN(dataloader,str(i))
        

********************** 23
Finaly positive keystroke images Data shape is (51, 5, 64, 64)
Finaly negative keystroke images Data shape is (18, 5, 64, 64)


 33%|███▎      | 1/3 [00:00<00:01,  1.30it/s]

(64, 5, 42, 42) HERE


100%|██████████| 3/3 [00:01<00:00,  1.54it/s]


********************** 24
Finaly positive keystroke images Data shape is (56, 5, 64, 64)
Finaly negative keystroke images Data shape is (15, 5, 64, 64)


...

100%|██████████| 4/4 [00:02<00:00,  1.54it/s]


********************** 74
Finaly positive keystroke images Data shape is (51, 5, 64, 64)
Finaly negative keystroke images Data shape is (16, 5, 64, 64)


 33%|███▎      | 1/3 [00:00<00:01,  1.28it/s]

(64, 5, 42, 42) HERE


100%|██████████| 3/3 [00:01<00:00,  1.54it/s]
